In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

/DATA/charchit.sharma/miniconda3/envs/diffusers_new/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_hf = GPT2LMHeadModel.from_pretrained('gpt2')
sd_hf = model_hf.state_dict()


for k, v in sd_hf.items():
    print(k,v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [ ]:
# what is the best way to communicate with the past tokens efficiently?
# it's by doing the averaging - it's a trick used in self-attention block and paper. 

import torch

# batches, time steps, channels
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [5]:
x

tensor([[[ 7.7506e-01, -1.6147e-01],
         [-2.6265e-03,  1.8576e+00],
         [-1.2308e+00, -9.0291e-01],
         [ 1.5273e-01,  4.9427e-01],
         [-2.6442e-02,  8.9439e-01],
         [ 2.1231e+00, -7.0380e-01],
         [ 5.9990e-01, -2.4542e-01],
         [-3.7558e-01, -1.4736e-01]],

        [[-3.5683e-01,  2.4422e+00],
         [-1.6937e-02, -1.4857e+00],
         [ 1.4758e+00,  6.7075e-01],
         [ 5.1657e-01, -3.1155e-01],
         [ 1.3056e+00,  2.7560e-01],
         [ 1.0211e-01,  1.3228e+00],
         [-1.0333e-01,  1.5477e+00],
         [ 2.5208e+00,  8.5146e-01]],

        [[-1.2445e+00, -6.6733e-01],
         [-5.1528e-01, -1.3041e+00],
         [ 6.4389e-01, -1.1951e+00],
         [ 2.9795e-01, -1.3119e-01],
         [-8.6628e-01, -1.9289e-01],
         [ 2.1652e+00, -8.2612e-01],
         [-4.7757e-01,  7.1056e-01],
         [-1.6694e+00, -1.0183e-01]],

        [[ 2.6349e-01,  2.1326e+00],
         [-1.3026e-02, -6.3189e-01],
         [ 4.7327e-01, -6.5539e-

In [7]:
xbow = torch.zeros((3,4,2))

In [8]:
xbow

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])

In [ ]:
temp1 = torch.randn(3,2,16)
temp2 = torch.randn(3,2,16)
 
(temp1 @ temp2.transpose(1,2)).shape

torch.Size([3, 2, 2])

In [13]:
temp1 = torch.randn(3,2,16)
temp2 = torch.randn(3,2,16)
 
# for each batch of 3 we have (2,16) shaped tensor, now (2,16) (2,16) cannot be multiplied just like that
# so we need to do transpose so that we match the dim of (2,16) (16,2) and then we can multiply them!!!!!!!!!!!!!

(temp1 @ temp2.transpose(1,2)).shape

torch.Size([3, 2, 2])

In [20]:
# math trick in self-attention

a = torch.tril(torch.ones(3,3))
print(a)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])


In [24]:
torch.sum(a, 1, keepdim=True)

tensor([[1.],
        [2.],
        [3.]])

In [38]:
a / torch.sum(a, 1, keepdim=False)

tensor([[1.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.0000],
        [1.0000, 0.5000, 0.3333]])

In [92]:

import torch

torch.manual_seed(42)

x = torch.randn(2, 5, 2)

B, T, C = x.shape

# Initialize xbow with zeros
xbow = torch.zeros((B, T, C))

# Compute the mean up to each time step t for each batch b
for b in range(B):
    for t in range(T):
        # select element from the batch and t+1 rows from it. 
        xprev = x[b, :t+1]  # Select elements up to time step t
        
        xbow[b, t] = torch.mean(xprev, 0)  # Compute mean along the 0th dimension
        print('what is this',xbow[b,t])

print("Original tensor x:")
print(x)
print("\nResulting tensor xbow:")
print(xbow)

what is this tensor([1.9269, 1.4873])
what is this tensor([ 1.4138, -0.3091])
what is this tensor([0.6898, 0.1534])
what is this tensor([0.7176, 0.5352])
what is this tensor([0.6452, 0.2908])
what is this tensor([-0.4934,  0.2415])
what is this tensor([-0.3625,  0.1416])
what is this tensor([-0.3255,  0.3810])
what is this tensor([-0.3216,  0.1868])
what is this tensor([-0.0966,  0.0252])
Original tensor x:
tensor([[[ 1.9269,  1.4873],
         [ 0.9007, -2.1055],
         [-0.7581,  1.0783],
         [ 0.8008,  1.6806],
         [ 0.3559, -0.6866]],

        [[-0.4934,  0.2415],
         [-0.2316,  0.0418],
         [-0.2516,  0.8599],
         [-0.3097, -0.3957],
         [ 0.8034, -0.6216]]])

Resulting tensor xbow:
tensor([[[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 0.6898,  0.1534],
         [ 0.7176,  0.5352],
         [ 0.6452,  0.2908]],

        [[-0.4934,  0.2415],
         [-0.3625,  0.1416],
         [-0.3255,  0.3810],
         [-0.3216,  0.1868],
        

In [52]:
x

tensor([[[-1.1786,  0.0139],
         [-0.6803, -1.1450],
         [-1.0232, -0.0286],
         [ 0.3011,  0.2395],
         [-0.8162,  2.2226]],

        [[ 0.0289, -0.1587],
         [ 0.3913,  0.0171],
         [-2.3095, -1.2788],
         [-0.4341,  0.2688],
         [ 0.9233,  0.4657]]])

In [ ]:
x[0,:3]

tensor([[-1.1786,  0.0139],
        [-0.6803, -1.1450],
        [-1.0232, -0.0286]])

In [93]:
wei = torch.tril(torch.ones(5, 5))
wei, wei.shape

(tensor([[1., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1.]]),
 torch.Size([5, 5]))

In [94]:
wei.sum(1, keepdim=True)

tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.]])

In [98]:
wei = wei / wei.sum(1, keepdim=True)

In [99]:
xbow2 = wei @ x 
xbow2

tensor([[[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 0.6898,  0.1534],
         [ 0.7176,  0.5352],
         [ 0.6452,  0.2908]],

        [[-0.4934,  0.2415],
         [-0.3625,  0.1416],
         [-0.3255,  0.3810],
         [-0.3216,  0.1868],
         [-0.0966,  0.0252]]])

In [100]:
torch.allclose(xbow, xbow2)

True

In [101]:
xbow[0], xbow2[0]

(tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 0.6898,  0.1534],
         [ 0.7176,  0.5352],
         [ 0.6452,  0.2908]]),
 tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 0.6898,  0.1534],
         [ 0.7176,  0.5352],
         [ 0.6452,  0.2908]]))

In [ ]:
t1 = wei.sum(1, keepdim=True)
t2 = wei.sum(1, keepdim=False)

t1.shape, t2.shape # t2.shape will be reduced by 1 dimension

(torch.Size([4, 1]), torch.Size([4]))

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


In [118]:
torch.zeros((3,3)).masked_fill(torch.tril(torch.ones(3,3)) == 0, float('-inf'))

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

In [119]:
import torch.nn.functional as F
F.softmax(torch.zeros((3,3)).masked_fill(torch.tril(torch.ones(3,3)) == 0, float('-inf')), dim=-1)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

Linear(in_features=2, out_features=8, bias=False)
